In [ ]:
import matplotlib.pyplot as plt
import os
import pandas as pd

In [24]:
output_dir = "graficos_adosamiento_edificio"
os.makedirs(output_dir, exist_ok=True)

In [25]:
nepal = pd.read_csv("eq2015.csv")
fuente_1 = "https://www.kaggle.com/datasets/sanskarnegi/nepal-earthquake-2015"

In [26]:
distritos = nepal["district_name"].unique()
distritos

array(['OKHALDHUNGA', 'SINDHULI', 'RAMECHHAP', 'DOLAKHA', 'SINDHUPALCHOK',
       'KAVRE', 'NUWAKOT', 'RASUWA', 'DHADING', 'MAKWANPUR', 'GORKHA'],
      dtype=object)

In [27]:
graficos_por_distrito = {}

In [44]:
for distrito in distritos:
    df_distrito = nepal[nepal["district_name"] == distrito]
    df_grouped = df_distrito.groupby(["position", "damage_grade"]).size().unstack()

    # Normalizar las filas para que cada barra tenga altura total 1
    df_normalized = df_grouped.div(df_grouped.sum(axis=1), axis=0)

    # Crear la figura
    fig, ax = plt.subplots(figsize=(7, 6))
    df_normalized.plot(kind='bar', stacked=True, colormap='viridis', ax=ax,legend=False)

    # Etiquetas
    plt.xlabel('Tipo de adosamiento')
    plt.ylabel('Número de edificios')
    plt.title(f'Distrito: {distrito}')
    ax.set_ylim(0, 1)

    for i, (idx, row) in enumerate(df_normalized.iterrows()):
        y_offset = 0
        for damage_grade, value in row.items():
            if value > 0:
                ax.text(
                    i + 0.5,
                    y_offset + value / 2, 
                    f"{value * 100:.1f}%", 
                    ha='right', 
                    va='center',
                    fontsize=7,
                    color='black'
                )
            y_offset += value
    # Guardar la imagen en memoria
    img_path = os.path.join(output_dir, f"{distrito}.png")
    plt.savefig(img_path, bbox_inches="tight", dpi=150)
    plt.close(fig)

    # Guardar ruta de la imagen
    graficos_por_distrito[distrito] = img_path